# Praca Domowa 2
Bartosz Siński

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from category_encoders import TargetEncoder
from category_encoders import HashingEncoder
from category_encoders import BinaryEncoder
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error

In [ ]:
df_trans = pd.read_csv("./src/allegro-api-transactions.csv")

In [ ]:
df_trans.head(5)

# 1.Kodowanie zmiennych kategorycznych

In [ ]:
df_trans['it_location'] = TargetEncoder().fit_transform(df_trans['it_location'],df_trans['price'])
df_trans

Przewagą target encoding nad one-hot encoding jest to, że one-hot encoding dla wielu zmiennych kategorycznych może w sposób znaczący zwiększać wymiarowość naszego zbioru danych. Będzie to szczególnie uciążliwe dla zbioru danych zawierających małą liczbę rekordów. 

In [ ]:
df_trans_onehot = pd.get_dummies(df_trans,prefix="",prefix_sep="", columns=['main_category'])
df_trans_onehot

In [ ]:
df_trans_hashing = HashingEncoder(cols='main_category').fit_transform(df_trans)
df_trans_hashing

In [ ]:
df_trans_binary = BinaryEncoder(cols='main_category').fit_transform(df_trans)
df_trans_binary

One-hot encoding każdej zmiennej kategorycznej przypisuje wartość 1 lub 0, co może spowodowac powstanie wielu nowych kolumn i zwiększenie wymiarowości naszego zbioru danych. Hash Encoder hashuje wartości naszych zmiennych kategorycznych za pomocą zer i jedynek w n-nowych wymiarach. n jest ustawiane przez użytkownika co pozwala na zmniejszego liczby nowo powstałych kolumn.  Jeżeli jednak damy za małe n dla liczby kategorii naszej zmiennej możemy spowodować, że różne kategorie będą miały ten sam klucz hashujący. W binary encoding na początku przekształcamy każdy kategorie na liczbę, a później zapisujemy ją z uzyciem nowych kolumn w formie binarnej. 

# 2. Uzupełnienie braków

Do tego ćwiczenia ograniczami liczbę rekordów, ponieważ czasy wykonywania niektórych poleceń były zbyt długie.

In [ ]:
df_trans_num = df_trans.loc[0:42000,['price','it_seller_rating','it_quantity']]
df_trans_num

In [ ]:
np.random.seed = 42
rows = np.random.randint(42000,size=4200)
df_trans_num_nan = df_trans_num.copy()
df_trans_num_nan.loc[rows,['it_seller_rating']] = np.nan 
df_trans_num_nan.info()

In [ ]:
df_trans_num0 = KNNImputer(n_neighbors=2, weights="uniform").fit_transform(df_trans_num_nan)
df_trans_num0 = pd.DataFrame(df_trans_num0, columns = ['price','it_seller_rating','it_quantity'])
mean_squared_error(df_trans_num[['it_seller_rating']],df_trans_num0[['it_seller_rating']],squared=False)

In [ ]:
results=[]
for i in range(10):
    rows = np.random.randint(42000,size=4200)
    df_trans_num_nan = df_trans_num.copy()
    df_trans_num_nan.loc[rows,['it_seller_rating']] = np.nan 
    df_trans_numn = KNNImputer(n_neighbors=5, weights="uniform").fit_transform(df_trans_num_nan)
    df_trans_numn = pd.DataFrame(df_trans_numn, columns = ['price','it_seller_rating','it_quantity'])
    results.append(mean_squared_error(df_trans_num[['it_seller_rating']],df_trans_numn[['it_seller_rating']], squared=False))
results

In [ ]:
results2=[]
for i in range(10):
    rows = np.random.randint(42000,size=4200)
    rows2 = np.random.randint(42000,size=4200)
    df_trans_num_nan = df_trans_num.copy()
    df_trans_num_nan.loc[rows,['it_seller_rating']] = np.nan 
    df_trans_num_nan.loc[rows2,['it_quality']] = np.nan 
    df_trans_numn = KNNImputer(n_neighbors=5, weights="uniform").fit_transform(df_trans_num_nan)
    df_trans_numn = pd.DataFrame(df_trans_numn, columns = ['price','it_seller_rating','it_quantity'])
    results2.append(mean_squared_error(df_trans_num[['it_seller_rating']],df_trans_numn[['it_seller_rating']],squared=False))
results2

In [ ]:
print("Odchylenie standardowe błędów wartości zmiennej it_seller_rating  dla: jednej zmiennej usuniętej="+str(np.std(results))+", dwóch zmiennych usuniętych="+str(np.std(results2)))
print("Średni błąd RMSE it_seller_rating dla: jednej zmiennej usuniętej="+str(np.mean(results))+", dwóch zmiennych usuniętych="+str(np.mean(results2)))

Wartości błędów są rzędu 10^4, czyli taki samo jak wartości it_seller_rating więc wypełnienie luk nie jest dokładne. Może być to spowodowane tym, że szukamy sąsiadów na podstawie tylko dwóch pozostałych zmiennych. Widzimy także na poniższym wykresie, że wartości są rozłożone nierównomiernie co także może powodować wysoki błąd metody najbliższych sąsiadów. Wynik są też dokładniejsze gdy zmienne na podstawie, których wypełniamy braki, także nie posiadają wartości None lub NaN. 

In [ ]:
sns.histplot(data=df_trans_numn,x="it_seller_rating")

Histogram błędów RMSE dwóch pomiarów, *results1* to eksperymenty gdzie brakowało jednej zmiennej, a *results2* to eksperymenty, gdzie brakowało dwóch zmiennych. 

In [ ]:
df_results=pd.DataFrame({'results1':results,'results2':results2})
df_results= pd.melt(df_results)
sns.histplot(data=df_results,x="value",hue="variable",multiple = "dodge")